In [ ]:
#import sys
#sys.path.insert(0, '/path/to/seir-visavis') # in order to be able to import from scripts.py

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scripts.client import VisAVisClient
from scripts.make_protocol import make_protocol

In [ ]:
from subplots_from_axsize import subplots_from_axsize

def plot_result(outfile, result, title=None):
    data = result.states
    
    fig, ax = subplots_from_axsize(
        1, 1, axsize=(20, 8),
        left=0., right=0., bottom=0., top=0.
    )

    data_selected = data.copy() #data[data['seconds'] < 4*60*60].copy()
    data_selected['E'] = data_selected['E'] > 0
    data_selected['I'] = data_selected['I'] > 0
    data_selected['R'] = data_selected['R'] > 0

    img_E = data_selected.groupby(['seconds', 'h'])['E'].mean().unstack().to_numpy().T
    img_I = data_selected.groupby(['seconds', 'h'])['I'].mean().unstack().to_numpy().T
    img_R = data_selected.groupby(['seconds', 'h'])['R'].mean().unstack().to_numpy().T

    img = img_E + img_I

    ax.imshow(
        img,
        cmap='gray',
        origin='lower',
        aspect='auto',
        interpolation='none',
    )

    ax.set_xlabel('time')
    ax.set_axis_off()
    
    if title is not None:
        ax.annotate(
            title, (0.5, 0.9),
            xycoords='axes fraction', va='center', ha='center',
            color='red', fontsize=32
        )

    fig.savefig(outfile)
    plt.close(fig)

In [ ]:
PARAMETERS_DEFAULT = {
  "c_rate": 1,
  "e_incr": 1,
  "i_incr": 1,
  "r_incr": 0.0667
}

In [ ]:
%%time
sim_num = 100
results = []
for channel_height in [7]:

    client = VisAVisClient(
        visavis_bin=f'/home/hombresabio/AI/Waves/visavis-seir/target/bins/vis-a-vis-{channel_height}',
    )

    for interval in [100]:

        protocol_file_path = make_protocol(
            pulse_intervals = [interval, 1500],
            duration=4,
            out_folder='./', #f'./interval-{interval}',
        )
        
        for sim in range(sim_num):
            result = client.run(
                parameters_json=PARAMETERS_DEFAULT,
                protocol_file_path= protocol_file_path,
            )
            results.append(result.states)

        #plot_result(
        #    f"./tkankony_{channel_height}_{interval}.png",
        #    result,
        #    title=f"channel height: {channel_height}, interval: {interval}"
        #)

# Visualization

In [ ]:
cols = 4
rows = 20
plt.figure(figsize=(4* cols, 4 * rows)) 
for i, df in enumerate(results[:rows]):
    ax_1 = plt.subplot(rows, cols, 4 * i + 1)
    img_E = df.groupby(['seconds', 'h'])['E'].sum().unstack().T
    img_I = df.groupby(['seconds', 'h'])['I'].sum().unstack().T
    img = img_E + img_I
    h_max = df['h'].max()
    h_min = df['h'].min() + 1 # at h = 0 we have empty cells blocking the signal
    
    ax_1.imshow(
        img.to_numpy(),
        cmap='gray',
        origin='lower',
        aspect='auto',
        interpolation='none',
    )
    ax_1.set_xlabel('time')
    ax_1.set_axis_off()

    ax_2 = plt.subplot(rows, cols, 4 * i + 2)
    roll_start = img.iloc[h_min].rolling(5).mean() # img indexed by position in channel 
    activations_start = roll_start[(roll_start > 0) & (roll_start.shift(-1) == 0)].reset_index()
    activations_start = activations_start[((activations_start.seconds - activations_start.seconds.shift(1) > 40)) | (activations_start.seconds.shift(1).isna())]
    plt.scatter(activations_start.seconds, activations_start[h_min], color = 'red')
    plt.plot(roll_start)
    ax_2.set_title(f"Pulses at start {len(activations_start)}")

    ax_3 = plt.subplot(rows, cols, 4 * i + 3)
    roll_finish = img.iloc[h_max].rolling(5).mean()
    activations_finish = roll_finish[(roll_finish > 0) & (roll_finish.shift(-1) == 0)].reset_index()
    activations_finish = activations_finish[((activations_finish.seconds - activations_finish.seconds.shift(1) > 40)) | (activations_finish.seconds.shift(1).isna()) ]
    plt.scatter(activations_finish.seconds, activations_finish[h_max], color = 'red')

    plt.plot(roll_finish)  
    ax_3.set_title(f"Pulses on finish {len(activations_finish)}")


    ax_4 = plt.subplot(rows, cols, 4 * i + 4)
    roll_total_infected = img.sum(axis=0).rolling(10).mean() # sum over all positions at given moment
    plt.plot(roll_total_infected)  
    ax_4.set_title(f"Infected cells moving average")


todo: Co dzieje się z drugim pulsem:
1. dociera
2. znika
3. rodzi (co i ile)<br>
a) Kiedy <br>
b) Jak daleko od 1 pulsu

# Analysis

In [ ]:
activations = []
stats = []
for i, df in enumerate(results):
    img_E = df.groupby(['seconds', 'h'])['E'].sum().unstack().T
    img_I = df.groupby(['seconds', 'h'])['I'].sum().unstack().T
    img = img_E + img_I

    roll_start = img.iloc[1].rolling(3).mean()
    activations_start = roll_start[(roll_start > 0) & (roll_start.shift(-1) == 0)]
    times_start = activations_start.index.values
    activations.extend([(time, 'start') for time in times_start])

    roll_finish = img.iloc[-1].rolling(3).mean()
    activations_finish = roll_finish[(roll_finish > 0) & (roll_finish.shift(-1) == 0)]
    times_finish = activations_finish.index.values
    activations.extend([(time, 'finish') for time in times_finish])

    if len(times_finish) < 2:
        times_finish = np.append(times_finish,[np.nan, np.nan])

    stats.append((len(activations_start), len(activations_finish), times_finish[0], times_finish[1]))

In [ ]:
df_activations = pd.DataFrame(activations, columns=['time', 'type'])
df_stats = pd.DataFrame(stats, columns = ['Start Activations', 'Finish Activations', 'First Finish', 'Second Finish'])

In [ ]:
plt.title("Time of arrival of signal at the end of a cannal")
sns.histplot(df_activations[df_activations.type == "finish"], bins = 100)

plt.axvline(x=df_stats['First Finish'].min(), color = 'g')
plt.axvline(x=df_stats['First Finish'].max(), color = 'g')

plt.axvline(x=df_stats['Second Finish'].min(), color = 'red')
plt.axvline(x=df_stats['Second Finish'].max(), color = 'red')

#plt.axvline(x=920, color = 'pink')
#df_stats.loc[df_stats['Second Finish'] > 920, 'Second Finish'] = np.nan

In [ ]:
p = df_stats['Second Finish'].value_counts().sum() / len(df_stats)
print(f"1. P dotarcia: {p:.2f}\n2. P zaniku/kolizji: {1-p:.2f}")

In [ ]:
df_second_fine = df_stats[(~df_stats['Second Finish'].isna()) & (~df_stats['First Finish'].isna())]

In [ ]:
plt.figure(figsize=(12, 4)) 
plt.subplot(1, 3, 1).set_title("Additional fronts - start line", fontsize = 10)
sns.histplot(df_second_fine['Start Activations'] - 2, stat='probability')
plt.subplot(1, 3, 2).set_title("Additional fronts - finish line", fontsize = 10)
sns.histplot(df_second_fine['Finish Activations'] - 2, stat='probability')
plt.subplot(1, 3, 3).set_title("Additional fronts - sum", fontsize = 10)
sns.histplot(df_second_fine['Start Activations'] + df_second_fine['Finish Activations'] - 4, stat='probability')
